In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from sklift.metrics import uplift_auc_score
from sklift.models import ClassTransformation
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

from torch.utils.data import Dataset
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader
import torch
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

from tqdm.notebook import tqdm

import re
from collections import Counter

import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

In [2]:
train = pd.read_parquet('data/train.parquet')
test = pd.read_parquet('data/test.parquet')
video_stat = pd.read_parquet('data/video_stat.parquet')

In [3]:
# Объединение train с video_stat по video_id
train = train.merge(video_stat, on='video_id', how='left')

# Объединение test с video_stat по video_id
test = test.merge(video_stat, on='video_id', how='left')

In [4]:
train = train[train["v_duration"] != 0]

In [5]:
def is_long_view(row):
    if row['v_duration'] > 300:
        return int(row['watchtime'] > 0.25 * row['v_duration'])
    else:
        return int(row['watchtime'] > 30)

train['long_view'] = train.apply(is_long_view, axis=1)

In [6]:
# categorical_features = ['region', 'city', 'title', 'description', 'author_id', 'category_id']
categorical_features = ['category_id', "author_id", "city", "region"]

# for col in categorical_features:
#     le = LabelEncoder()
#     train[col] = le.fit_transform(train[col].astype(str))
#     test[col] = le.transform(test[col].astype(str))

In [7]:
features1 = [
    'v_total_comments',
    'v_year_views', 'v_month_views', 'v_week_views', 'v_day_views',
    'v_likes', 'v_dislikes', 'v_cr_click_like_7_days',
    'v_cr_click_dislike_7_days', 'v_cr_click_vtop_7_days',
    'v_cr_click_long_view_7_days', 'v_cr_click_comment_7_days',
    'v_cr_click_like_30_days', 'v_cr_click_dislike_30_days',
    'v_cr_click_vtop_30_days', 'v_cr_click_long_view_30_days',
    'v_cr_click_comment_30_days', 'v_cr_click_like_1_days',
    'v_cr_click_dislike_1_days', 'v_cr_click_vtop_1_days',
    'v_cr_click_long_view_1_days', 'v_cr_click_comment_1_days',
    'v_avg_watchtime_1_day', 'v_avg_watchtime_7_day',
    'v_avg_watchtime_30_day',
    'v_category_popularity_percent_7_days',
    'v_category_popularity_percent_30_days', 'v_long_views_1_days',
    'v_long_views_7_days', 'v_long_views_30_days',
    'category_id', "author_id", "city", "region"
]

features2 = [
    'v_total_comments',
    'v_year_views', 'v_month_views', 'v_week_views', 'v_day_views',
    'v_likes', 'v_dislikes', 'v_duration', 'v_cr_click_like_7_days',
    'v_cr_click_dislike_7_days', 'v_cr_click_vtop_7_days',
    'v_cr_click_long_view_7_days', 'v_cr_click_comment_7_days',
    'v_cr_click_like_30_days', 'v_cr_click_dislike_30_days',
    'v_cr_click_vtop_30_days', 'v_cr_click_long_view_30_days',
    'v_cr_click_comment_30_days', 'v_cr_click_like_1_days',
    'v_cr_click_dislike_1_days', 'v_cr_click_vtop_1_days',
    'v_cr_click_long_view_1_days', 'v_cr_click_comment_1_days',
    'v_avg_watchtime_1_day', 'v_avg_watchtime_7_day',
    'v_avg_watchtime_30_day', 'v_frac_avg_watchtime_1_day_duration',
    'v_frac_avg_watchtime_7_day_duration',
    'v_frac_avg_watchtime_30_day_duration',
    'v_category_popularity_percent_7_days',
    'v_category_popularity_percent_30_days', 'v_long_views_1_days',
    'v_long_views_7_days', 'v_long_views_30_days',
    'category_id', "author_id", "city", "region"
]

In [16]:
X = train[features1]
y = train['long_view']

# X_train, X_valid, y_train, y_valid = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y
# )

model1 = CatBoostClassifier(
    depth=6, iterations=400, random_state=42, cat_features=categorical_features, verbose=100, thread_count=4
)
# model1 = LogisticRegression(max_iter=1000)
# model1 = RandomForestClassifier(n_estimators=100)
model1.fit(X, y)

Learning rate set to 0.5
0:	learn: 0.5925109	total: 8.36s	remaining: 55m 36s
100:	learn: 0.5444419	total: 12m 9s	remaining: 35m 59s
200:	learn: 0.5431428	total: 24m 41s	remaining: 24m 26s
300:	learn: 0.5423321	total: 36m 15s	remaining: 11m 55s
399:	learn: 0.5417864	total: 50m 33s	remaining: 0us


In [17]:
X = train[features2]
y = train['long_view']

# X_train, X_valid, y_train, y_valid = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y
# )

model2 = CatBoostClassifier(
    depth=6, iterations=400, random_state=42, cat_features=categorical_features, verbose=100, thread_count=4
)
# model2 = LogisticRegression(max_iter=1000)
# model2 = RandomForestClassifier(n_estimators=100)
model1.fit(X, y)

Learning rate set to 0.5
0:	learn: 0.5892924	total: 10.1s	remaining: 1h 7m 18s


KeyboardInterrupt: 

In [18]:
y_pred = model1.predict(X_valid[features1])
f1 = f1_score(y_valid, y_pred)
print(f'F1-Score на валидационной выборке: {f1:.4f}')

F1-Score на валидационной выборке: 0.7680


In [11]:
y_pred = model2.predict(X_valid[features2])
f1 = f1_score(y_valid, y_pred)
print(f'F1-Score на валидационной выборке: {f1:.4f}')

F1-Score на валидационной выборке: 0.7539


# Тест

In [19]:
test1 = test[test["v_duration"] == 0]
test1 = test1[features1]
test2 = test[test["v_duration"] != 0]
test2 = test2[features1]
test1["target"] = model1.predict(test1[features1])
test2["target"] = model1.predict(test2[features1])
test1["target"] = test1[["target"]]
test2["target"] = test2[["target"]]
test_pred = pd.concat([test1, test2]).sort_index()["target"]

submission = pd.DataFrame({
    'target': test_pred
}, index=test.index)

submission.to_csv('data/submission.csv', index=False)